In [22]:
import torch
user_label=torch.tensor([0],dtype=torch.long)
user_emb=torch.nn.Embedding(5,64)
user_emb(user_label)

tensor([[ 0.8963,  0.0559,  1.1192, -1.5954,  1.1224, -1.9931,  0.1907,  0.8361,
          0.5467, -1.1453, -2.0601, -0.1190, -1.0320, -0.7400, -0.0785,  0.9139,
          1.5257,  0.1175,  0.2601,  1.8335,  1.5002,  2.9357, -0.3135,  1.5546,
         -0.6345,  0.9917, -0.9781,  1.0796, -0.0789,  0.2662,  0.4885,  1.0763,
          0.5281,  0.5225, -0.9784, -1.4573, -0.4711,  1.5919, -1.6874, -0.8403,
          0.9285, -1.1401,  1.7858,  0.6263,  0.0931, -0.4849,  1.0473,  1.0102,
         -1.2043, -1.1656,  0.0785, -1.2349,  2.1460,  0.4609, -0.8146, -0.2024,
         -0.2438, -0.3257,  0.4290,  0.4453,  0.8743,  0.1126,  0.6620, -0.0671]],
       grad_fn=<EmbeddingBackward0>)

In [11]:
recipe_label=torch.tensor([0,1,459],dtype=torch.long)
recipe_emb=torch.nn.Embedding(460,64)
recipe_emb(user_label)

tensor([[ 1.2632,  0.0822, -0.2786, -0.8591,  1.1829, -0.6288, -1.3667, -0.7359,
          1.6283, -0.7824,  0.0457, -1.1484, -0.1239, -0.5019, -1.8768,  0.4935,
         -0.1285,  1.1686,  0.3725, -1.1335, -1.0557,  0.7397, -0.2655, -0.4356,
          0.4855, -0.7807,  0.1209, -1.5405, -1.1465, -0.1590, -0.2357,  0.2347,
          0.5258, -0.8524,  0.4881,  0.9015, -0.6352, -1.0535,  2.1813, -1.3436,
          1.2460, -0.5376, -0.0948, -0.1544,  0.2819,  0.2734, -0.8474, -1.0595,
          0.8982, -0.0245, -0.9605,  1.7596,  0.0109, -0.6126, -0.5688, -0.6833,
         -0.6252, -1.9870,  0.5176, -0.8208, -0.4003,  0.8671, -0.5050, -0.4994],
        [-0.2815,  1.5993,  0.9366, -1.1613,  1.2990, -1.4912,  1.2306, -1.7752,
         -0.3834, -0.3045, -1.9369,  1.0323, -0.8659, -0.0808,  1.9767,  1.2064,
         -1.3468, -0.5866, -0.0023,  1.6303, -0.7666,  0.3173,  1.8220, -0.3435,
          0.7542, -1.5753, -1.1189,  0.3136,  0.1396,  1.6543,  0.5181,  0.6847,
         -1.0639, -0.6996, 

In [131]:
import torch
import pandas as pd
def get_category(s:str):
    if s=='无':
        return []
    temp=s.split(';')
    if '' in temp:
        temp.remove('')
    return temp
data=pd.read_csv('final.csv')
user_emb=torch.nn.Embedding(5,64).cuda()
recipe_emb=torch.nn.Embedding(len(data),64).cuda()
opt_recipe=torch.optim.Adam(params=recipe_emb.parameters(),lr=0.01)
opt_user=torch.optim.Adam(params=user_emb.parameters(),lr=0.01) 
ground_true=torch.zeros(size=(len(data),5)).cuda()
for i in range(len(data)):
    #print(data.at[i,'name'])
    less=get_category(data.at[i,'less'])
    no=get_category(data.at[i,'no'])  
    more=get_category(data.at[i,'more'])  
    if '高血脂人群' in less or '高血脂人群' in no:
        ground_true[i][0]=-1
    else :
        ground_true[i][0]=1
    if '高血压人群' in less or '高血压人群' in no:
        ground_true[i][1]=-1
    else :
        ground_true[i][1]=1   
    if '高血糖人群' in less or '高血糖人群' in no:
        ground_true[i][2]=-1
    else :
        ground_true[i][2]=1 
    if '减肥人群' in less or '减肥人群' in no:
        ground_true[i][3]=-1
    else :
        ground_true[i][3]=1      
    if '儿童' in less or '儿童' in no:
        ground_true[i][4]=-1
    else :
        ground_true[i][4]=1
#print(ground_true)
for epoch in range(10000):
    recipe=torch.tensor([i for i in range(len(data))],dtype=torch.long).cuda()
    recipe=recipe_emb(recipe)
    recipe_normed=torch.nn.functional.normalize(recipe)
    user=torch.tensor([0,1,2,3,4],dtype=torch.long).cuda()
    user=user_emb(user)
    user_normed=torch.nn.functional.normalize(user)
    
    res=torch.mm(recipe_normed,user_normed.T)   
    loss_func=torch.nn.MSELoss()
    loss=loss_func(res,ground_true)
    #print(loss.item())
    loss.backward()
    #print(user_emb.weight.grad)
    opt_recipe.step()
    opt_recipe.zero_grad()
    opt_user.step()
    opt_user.zero_grad()
print(loss.item())
print(res)

0.222104012966156
tensor([[ 0.7007,  0.3696,  0.5564,  0.6203, -0.1227],
        [ 0.7215,  0.8254,  0.5628,  0.6911,  0.6717],
        [ 0.2703, -0.3149, -0.4678,  0.4421, -0.5484],
        ...,
        [ 0.7215,  0.8253,  0.5628,  0.6911,  0.6716],
        [ 0.7446,  0.7244, -0.3893,  0.5745,  0.6527],
        [ 0.2706, -0.3147, -0.4678,  0.4424, -0.5482]], device='cuda:0',
       grad_fn=<MmBackward0>)


In [129]:
recipe=torch.tensor([2],dtype=torch.long).cuda()
recipe=recipe_emb(recipe)
recipe_normed=torch.nn.functional.normalize(recipe)
user=torch.tensor([0,1,2,3,4],dtype=torch.long).cuda()
user=user_emb(user)
user=torch.nn.functional.normalize(user)
res=torch.mul(recipe_normed,user).sum(axis=1)
res

tensor([ 0.2706, -0.3148, -0.4676,  0.4421, -0.5480], device='cuda:0',
       grad_fn=<SumBackward1>)

In [162]:
torch.save(recipe_emb.state_dict(),'recipe_emb.emb')
torch.save(user_emb.state_dict(),'user_emb.emb')

看菜谱之间的相似性

In [153]:
v1=recipe_emb(torch.tensor([10],dtype=torch.long).cuda())
v2=recipe_emb(torch.tensor([11],dtype=torch.long).cuda())
v1=torch.nn.functional.normalize(v1)
v2=torch.nn.functional.normalize(v2)
torch.mul(v1,v2).sum()

tensor(1., device='cuda:0', grad_fn=<SumBackward0>)

加载权重

In [159]:
new_emb=torch.nn.Embedding(460,64)
new_emb.load_state_dict(torch.load('recipe_emb.emb'))
new_emb.weight.data

tensor([[-0.2021, -0.8387,  0.0320,  ...,  1.4194, -0.7064, -0.5632],
        [ 0.7451, -1.2412,  0.9944,  ...,  0.6057,  0.9759, -1.1060],
        [-0.6803,  1.6156, -1.2770,  ...,  0.4323, -1.3780,  1.0010],
        ...,
        [ 0.6402, -1.0662,  0.8539,  ...,  0.5203,  0.8383, -0.9502],
        [ 0.8522,  0.9789,  0.9829,  ..., -1.0503,  1.1707, -0.6890],
        [-0.4701,  1.1162, -0.8820,  ...,  0.3000, -0.9524,  0.6916]])

In [175]:
user_emb_list=[]
user_emb_list.append(user_emb(torch.tensor(0,dtype=torch.long).cuda()))
user_emb_list.append(user_emb(torch.tensor(1,dtype=torch.long).cuda()))
user_emb_list.append(user_emb(torch.tensor(2,dtype=torch.long).cuda()))
print(user_emb_list)
print(torch.vstack(user_emb_list).mean(axis=0))

[tensor([ 0.1548,  0.2174,  0.7350,  0.0638,  1.4656, -0.2658,  0.2731,  0.4702,
        -0.1132, -0.0445, -0.5553, -0.9214, -0.2566, -0.9001,  1.3749, -0.7419,
        -0.6950, -1.3102,  0.0526, -0.8809, -0.8346, -0.8057, -1.0165, -1.0726,
        -0.0521, -2.1990,  1.3979, -1.1863,  1.0047, -0.5018, -0.4555,  0.9257,
        -0.1985,  1.9617,  0.3995,  1.8863,  0.1302, -1.5533,  0.2180, -0.8187,
         0.3585, -0.2220,  1.4450, -0.0971, -1.3361, -0.0231,  1.5804,  1.8479,
        -0.8569, -1.4803,  0.3215,  0.6954, -1.5086, -0.2207,  0.4640, -1.6423,
        -0.7121,  0.4180, -0.7345, -1.2085, -0.4349,  0.2951,  0.1471, -0.6832],
       device='cuda:0', grad_fn=<EmbeddingBackward0>), tensor([ 7.8012e-01, -2.8745e-01,  1.5005e+00, -6.6879e-02,  9.6267e-01,
        -6.3299e-01, -1.0989e-01, -9.0523e-01,  1.0431e+00, -2.9850e-01,
        -6.8346e-01, -2.7747e-01,  2.1435e-01, -5.4605e-01,  1.6696e+00,
        -7.9392e-01,  9.0046e-02, -4.2985e-01, -2.9805e-01, -4.5940e-01,
        -1.

用户标签：高血脂人群、高血压人群、高血糖人群、减肥人群、儿童

In [15]:
import torch
import pandas as pd
kouwei_list=['酸辣味', '酸甜味', '家常味', '酱香味', '香辣味', '咸鲜味', '甜味', '鱼香味', '麻辣味',  '咖喱味', '茄汁味', '豆瓣味', '黑椒味', '蒜香味', '葱香味', '果味', '椒麻味','五香味', '奶香味','其它口味']
data=pd.read_csv('final.csv')
user_kouwei_emb=torch.nn.Embedding(len(kouwei_list),128)
recipe_kouwei_emb=torch.nn.Embedding(len(data),128)
opt_recipe=torch.optim.Adam(params=recipe_kouwei_emb.parameters(),lr=0.02)
opt_user=torch.optim.Adam(params=user_kouwei_emb.parameters(),lr=0.02) 
ground_true=torch.zeros(size=(len(data),len(kouwei_list)))
for i in range(len(data)):
    ind=kouwei_list.index(data.kouwei[i])
    ground_true[i][ind]=1
print(ground_true)
for epoch in range(1000):
    recipe=torch.tensor([i for i in range(len(data))],dtype=torch.long)
    recipe=recipe_kouwei_emb(recipe)
    recipe_normed=torch.nn.functional.normalize(recipe)
    user=torch.tensor([i for i in range(len(kouwei_list))],dtype=torch.long)
    user=user_kouwei_emb(user)
    user_normed=torch.nn.functional.normalize(user)
    res=torch.mm(recipe_normed,user_normed.T)
    loss_func=torch.nn.MSELoss()
    loss=loss_func(res,ground_true)
    #print(loss.item())
    loss.backward()
    #print(res)
    opt_recipe.step()
    opt_recipe.zero_grad()
    opt_user.step()
    opt_user.zero_grad()
print(loss.item())
print(res)
torch.save(recipe_kouwei_emb.state_dict(),'recipe_kouwei_emb.emb')
torch.save(user_kouwei_emb.state_dict(),'user_kouwei_emb.emb')

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
3.6329809063317953e-06
tensor([[ 9.9351e-01,  2.7703e-05,  1.7355e-06,  ..., -3.4637e-05,
         -2.6097e-05,  4.2713e-05],
        [-1.9602e-05,  9.9206e-01, -6.8982e-06,  ..., -5.6568e-05,
          9.6765e-06,  3.7320e-05],
        [ 8.7459e-06, -3.3227e-05,  9.9136e-01,  ...,  1.6570e-05,
          1.7419e-05,  5.7002e-08],
        ...,
        [-3.3596e-05,  8.1294e-06,  9.9099e-01,  ...,  4.3797e-05,
         -6.9584e-05, -1.1873e-05],
        [-3.9509e-05, -1.8039e-05,  9.9112e-01,  ...,  3.2138e-05,
          2.4724e-05, -3.0104e-05],
        [-2.9106e-05,  1.7228e-04,  6.6824e-05,  ...,  2.3402e-05,
          9.8202e-06,  1.0527e-04]], grad_fn=<MmBackward0>)
